In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def backtest(df, targetLtv, apr, upfrontFee=.0, loanTenorInDays=30, borrowsPerMonth=1000000., utilizationPerMonth=.9, ltvAdjTol=.05):
    df["VAULT-MAX-LOAN-PER-COLL"] = None
    df["VAULT-MAX-LOAN-PER-COLL-UPDATE"] = False
    df["VAULT-LTV"] = None

    df["PRICE-CHG"] = df["CLOSE"].pct_change(periods=-1)
    df["BORROWER-LOAN-AMOUNT"] = None
    df["BORROWER-LOAN-AMOUNT-PER-COLL"] = None
    df["BORROWER-REPAY-AMOUNT"] = None
    df["BORROWER-COLL-RECLAIMABLE"] = None
    df["BORROWER-EXPIRY-DATE"] = None
    df["BORROWER-PRICE-AT-EXPIRY"] = None
    df["BORROWER-WILL-REPAY"] = None

    df["TREASURY-LENT"] = 0
    df["TREASURY-UPFRONT-FEE"] = 0
    df["TREASURY-UPFRONT-FEE-CUM"] = 0
    df["TREASURY-UPFRONT-FEE-CUM-MtM"] = 0
    
    df["TREASURY-EXPIRING-LOANS-ARE-REPAID"] = None
    df["TREASURY-REPAYMENTS"] = 0
    df["TREASURY-NET-LOAN-TOKEN"] = 0
    df["TREASURY-NET-LOAN-TOKEN-CUM-MtM"] = 0
    
    df["TREASURY-UNCLAIMED-COLL-TOKEN"] = 0
    df["TREASURY-UNCLAIMED-COLL-TOKEN-CUM"] = 0
    df["TREASURY-UNCLAIMED-COLL-TOKEN-CUM-MtM"] = 0

    df["TREASURY-ROI-PER-LOAN"] = None
    df["TREASURY-PNL"] = 0

    treasuryRequiredLiquidity = borrowsPerMonth/utilizationPerMonth
    n = len(df)
    i = 1
    loanAmount = borrowsPerMonth / 30.
    maxLoanPerColl = targetLtv * df["CLOSE"][n-1]
    while i+loanTenorInDays < n:
        _, spotPrice = df["DATE"][n-i], df["CLOSE"][n-i]
        ltv = maxLoanPerColl / spotPrice
        if (abs(ltv - targetLtv) > ltvAdjTol):
            maxLoanPerColl = targetLtv * spotPrice
            ltv = maxLoanPerColl / spotPrice
            df["VAULT-MAX-LOAN-PER-COLL-UPDATE"].at[n-i] = True
        collSend = loanAmount / maxLoanPerColl / (1-upfrontFee)
        collReclaimable = collSend * (1-upfrontFee)
        collUpfrontFee = collSend * upfrontFee
        loanAmountPerColl = loanAmount / collReclaimable
        idxAtExpiry = n-i-loanTenorInDays-1
        expiryDate, priceAtExpiry = (df["DATE"][idxAtExpiry], df["CLOSE"][idxAtExpiry]) 
        repaymentAmount = loanAmount*(1+apr*loanTenorInDays/365.)
        borrowerWillRepay = collReclaimable*priceAtExpiry > repaymentAmount 
        
        df["VAULT-MAX-LOAN-PER-COLL"].at[n-i] = maxLoanPerColl
        df["VAULT-LTV"].at[n-i] = ltv

        df["BORROWER-LOAN-AMOUNT"].at[n-i] = loanAmount
        df["BORROWER-LOAN-AMOUNT-PER-COLL"].at[n-i] = loanAmountPerColl
        df["BORROWER-REPAY-AMOUNT"].at[n-i] = repaymentAmount
        df["BORROWER-COLL-RECLAIMABLE"].at[n-i] = collReclaimable
        df["BORROWER-EXPIRY-DATE"].at[n-i] = expiryDate
        df["BORROWER-PRICE-AT-EXPIRY"].at[n-i] = priceAtExpiry
        df["BORROWER-WILL-REPAY"].at[n-i] = borrowerWillRepay

        df["TREASURY-LENT"].at[n-i] = loanAmount
        df["TREASURY-UPFRONT-FEE"].at[n-i] = collUpfrontFee

        df["TREASURY-EXPIRING-LOANS-ARE-REPAID"].at[idxAtExpiry] = borrowerWillRepay
        df["TREASURY-REPAYMENTS"].at[idxAtExpiry] = repaymentAmount if borrowerWillRepay else 0
        df["TREASURY-NET-LOAN-TOKEN"].at[idxAtExpiry] = repaymentAmount - loanAmount if borrowerWillRepay else -loanAmount
        df["TREASURY-UNCLAIMED-COLL-TOKEN"].at[idxAtExpiry] = 0 if borrowerWillRepay else collReclaimable

        i += 1

    # cumulative coll token amounts
    df["TREASURY-UPFRONT-FEE-CUM"] = df.loc[::-1, "TREASURY-UPFRONT-FEE"].cumsum()[::-1]
    df["TREASURY-UNCLAIMED-COLL-TOKEN-CUM"] = df.loc[::-1, "TREASURY-UNCLAIMED-COLL-TOKEN"].cumsum()[::-1]
    
    # cumulative MtM values
    df["TREASURY-NET-LOAN-TOKEN-CUM-MtM"] = df.loc[::-1, "TREASURY-NET-LOAN-TOKEN"].cumsum()[::-1]
    df["TREASURY-UPFRONT-FEE-CUM-MtMM"] = df["TREASURY-UPFRONT-FEE-CUM"]*df["CLOSE"]
    df["TREASURY-UNCLAIMED-COLL-TOKEN-CUM-MtM"] = df["TREASURY-UNCLAIMED-COLL-TOKEN-CUM"]*df["CLOSE"]
    df["TREASURY-TOTAL-VALUE-CUM-MtM"] = df["TREASURY-NET-LOAN-TOKEN-CUM-MtM"] + df["TREASURY-UPFRONT-FEE-CUM-MtMM"] + df["TREASURY-UNCLAIMED-COLL-TOKEN-CUM-MtM"]
    treasuryFinalPnL = df["TREASURY-NET-LOAN-TOKEN-CUM-MtM"][0] + df["TREASURY-UPFRONT-FEE-CUM-MtMM"][0] + df["TREASURY-UNCLAIMED-COLL-TOKEN-CUM-MtM"][0]
    treasuryRoI = treasuryFinalPnL/treasuryRequiredLiquidity
    print(treasuryFinalPnL, treasuryRoI)
    res = {
        "from": df["DATE"][n-1],
        "to": df["DATE"][0],
        "initPrice": df["CLOSE"][n-1],
        "finalPrice": df["CLOSE"][0],
        "treasuryRequiredLiquidity": treasuryRequiredLiquidity,
        "treasuryFinalPnL": treasuryFinalPnL,
        "treasuryRoI": treasuryRoI
    }

    df.to_csv("backtest-result.csv")
    
    return df, res

def currency(x, pos):
    return '${:,}'.format(int(x))

def pct(x, pos):
    return '{:.1%}'.format(x)

if __name__ == "__main__":
    df = pd.read_excel('gohm-price-history.xlsx', header=0)

    # only apr and high ltv
    targetLtv, apr = 0.7, .08
    upfrontFee = .0
    
    loanTenorInDays = 30
    borrowsPerMonth = 100000
    utilizationPerMonth = .9
    ltvAdjTol = .05
    df, res = backtest(df, targetLtv, apr, upfrontFee, loanTenorInDays, borrowsPerMonth, utilizationPerMonth, ltvAdjTol)    

    print("MYSO V1.1 VAULT FOR OLYMPUS DAO - BACKTEST")
    print("targetLtv={:.2f}%, APR={:.2f}%, upfrontFee={:.2f}%, tenor={}d, borrowsPerMonth=${:0,.0f}, liquidityInVault=${:0,.0f}, utilizationPerMonth={:.2f}%, ltvAdjTol={:.2f}%)".format(targetLtv*100, apr*100, upfrontFee*100, loanTenorInDays, borrowsPerMonth, res["treasuryRequiredLiquidity"], utilizationPerMonth*100, ltvAdjTol*100))
    print("backtest from={}".format(res["from"]))
    print("backtest to={}".format(res["to"]))
    print("treasuryRequiredLiquidity=${:0,.0f}".format(res["treasuryRequiredLiquidity"]))
    print("treasuryFinalPnL=${:0,.0f}".format(res["treasuryFinalPnL"]))
    print("RoI={:.2f}%".format(res["treasuryRoI"]*100))

    fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, sharex=True, subplot_kw=dict(frameon=False), figsize=[32, 28])

    plt.subplots_adjust(hspace=.0)
    ax1.grid(visible=True, which='major', color='grey', linestyle=':')
    ax2.grid(visible=True, which='major', color='grey', linestyle=':')
    ax3.grid(visible=True, which='major', color='grey', linestyle=':')
    ax4.grid(visible=True, which='major', color='grey', linestyle=':')

    # price and max. loanable amount per coll.
    ax1.plot(df["DATE"], df["CLOSE"], '-', color="black", label="collateral price")
    ax1.plot(df["DATE"], df["BORROWER-LOAN-AMOUNT-PER-COLL"], '-', label="Max. Loan per Coll.", color="blue")
    ax1.plot(df["DATE"][df["VAULT-MAX-LOAN-PER-COLL-UPDATE"] == True], df["BORROWER-LOAN-AMOUNT-PER-COLL"][df["VAULT-MAX-LOAN-PER-COLL-UPDATE"] == True], 'o', label="Max. Loan per Coll. Adjustment", color="orange", markersize=12)

    ax1.set_title('MYSO Vault for gOHM - Backtest \n (targetLtv={:.2f}%, APR={:.2f}%, upfrontFee={:.2f}%, tenor={}d, \n borrowsPerMonth=\${:0,.0f}, liquidityInVault=\${:0,.0f}, utilizationPerMonth={:.2f}%, \n ltvAdjTol={:.2f}%)'.format(targetLtv*100, apr*100, upfrontFee*100, loanTenorInDays, borrowsPerMonth, res["treasuryRequiredLiquidity"], utilizationPerMonth*100, ltvAdjTol*100))
    ax1.set_ylabel("Price")
    ax1.get_xaxis().tick_bottom()
    ax1.get_yaxis().tick_left()
    ax1.yaxis.set_major_formatter(currency)
    box = ax1.get_position()
    ax1.set_position([box.x0, box.y0 + box.height * 0.1, box.width, box.height * 0.9])
    ax1.legend(loc='upper center', bbox_to_anchor=(1.2, 1), fancybox=True, shadow=True, ncol=1)
    
    # LTV
    ax2.set_title("LTV")
    ax2.vlines(df["DATE"][df["VAULT-LTV"]>0], df["VAULT-LTV"][df["VAULT-LTV"]>0]*0, df["VAULT-LTV"][df["VAULT-LTV"]>0], alpha=0.8)
    ax2.set_ylabel("LTV")
    ax2.get_xaxis().tick_bottom()
    ax2.get_yaxis().tick_left()
    ax2.yaxis.set_major_formatter(pct)
    
    # Cum. RoI
    ax3.set_title("Cumulative RoI (MtM)")
    tmp = df["TREASURY-TOTAL-VALUE-CUM-MtM"] / res["treasuryRequiredLiquidity"]
    ax3.fill_between(df["DATE"],tmp, '-', color="black", label="Cum. Total RoI (MtM)", alpha=0.5)
    ax3.yaxis.set_major_formatter(pct)
    ax3.set_yticks([tmp.min(), 0, tmp.max()])
    ax3.set_ylabel("RoI")
    ax3.get_xaxis().tick_bottom()
    ax3.get_yaxis().tick_left()
    
    # RoI Constituens
    ax4.set_title("Cumulative RoI Breakdown (MtM)")
    tmp1 = df["TREASURY-NET-LOAN-TOKEN-CUM-MtM"] / res["treasuryRequiredLiquidity"]
    tmp2 = df["TREASURY-UPFRONT-FEE-CUM-MtMM"] / res["treasuryRequiredLiquidity"]
    tmp3 = df["TREASURY-UNCLAIMED-COLL-TOKEN-CUM-MtM"] / res["treasuryRequiredLiquidity"]
    ax4.fill_between(df["DATE"], tmp1, '-', color="blue", label="RoI from Repayments/Defaults (MtM)", alpha=0.2)
    ax4.fill_between(df["DATE"], tmp2, '-', color="red", label="RoI from Upfront Fees (MtM)", alpha=0.2)
    ax4.fill_between(df["DATE"], tmp3, '-', color="green", label="RoI from Unclaimed Coll. (MtM)", alpha=0.2)
    ax4.plot(df["DATE"][df["TREASURY-EXPIRING-LOANS-ARE-REPAID"] == False], df["TREASURY-NET-LOAN-TOKEN-CUM-MtM"][df["TREASURY-EXPIRING-LOANS-ARE-REPAID"] == False] / res["treasuryRequiredLiquidity"], 'o', label="Default", color="red", markersize=12)
    
    ax4.set_yticks([min(tmp1.min(), tmp2.min(), tmp3.min()), 0., max(tmp1.max(), tmp2.max(), tmp3.max())])
    box = ax4.get_position()
    ax4.set_position([box.x0, box.y0 + box.height * 0.1, box.width, box.height * 0.9])
    ax4.legend(loc='upper center', bbox_to_anchor=(1.2, 1), fancybox=True, shadow=True, ncol=1)
    
    ax4.set_ylabel("RoI")
    ax4.get_xaxis().tick_bottom()
    ax4.get_yaxis().tick_left()
    ax4.yaxis.set_major_formatter(pct)
    
    plt.setp(ax4.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    fig.tight_layout(h_pad=2.8, w_pad=1)
    
    plt.show()
    fig.savefig('backtest-res-ltv={}%-apr={}%-upfrontFee={}bps-tenor={}d-borrowsPerMonth={}-ltvAdjTol={}.png'.format(int(targetLtv*100), int(apr*100), int(upfrontFee*10000), loanTenorInDays, borrowsPerMonth, int(ltvAdjTol*100)), dpi=fig.dpi, bbox_inches='tight')